# User ID Mining (USA)

## Modules

In [1]:
# Data Wrangling 
import pandas as pd
import numpy as np
import sys
sys.path.insert(1, r"Q:\IlMIoDrive\magistrale\1anno\terzo_periodo\MAS\analysis")

# Custom Modules
import Econophysics as Ec
import credentials as cr

# Reload Custom Modules
from importlib import reload
Ec  = reload(Ec)
cr = reload(cr)

# Twitter Scraping
import tweepy

# Basic Utilities
import time
import requests
import pickle

## Streamer Class

In [2]:
# Create a StreamListener 
class MyStreamListener(tweepy.StreamListener):
    def __init__(self,run_name, save_path):
        super().__init__()
        self.tweets_arrived = 0
        self.keep_alive_arrived = 0
        self.users_ids = set()
        self.tweets = []
        self.save_path = save_path+"\\"+run_name+".txt"
        
    def on_status(self, status):
        # Set time 
        #t = time.localtime()
        #current_time = time.strftime("%H:%M:%S", t)
        self.tweets_arrived += 1
        hashtags =  [dct["text"] for dct in status.entities["hashtags"]]
        #print("\nStatus arrived at", current_time)
        #print (status.text)
        
        self.users_ids.add(status.user.id)                     # add author id
        #print("AUTHOR ID:", status.user.id)
        #self.hashtags.append(tweet.entities["hashtags"])
        
        
#         if status.in_reply_to_user_id is not None:
#             #print("REPLIED ID:", status.in_reply_to_user_id)
#             self.user_ids.add(status.in_reply_to_user_id)     # add replied id
        
        try:
            retweeted_user_id  = status.retweeted_status.user.id
            #retweeted_user_id = retweeted_status
            self.tweets.append({"user_id":status.user.id, "hashtags":hashtags, "retweeted_user_id":retweeted_user_id})
        except AttributeError:
            self.tweets.append({"user_id":status.user.id, "hashtags":hashtags, "retweeted_user_id":None})
            pass
        
        #np.save("/Users/Pit/GitHub/Econophysics/Project/TwitterMonitor/Streamer_USA/UserIDs/user_ids.npy", np.array(list(self.user_ids))) # PIETRO 1 
        #np.save("/Users/pietromonticone/github/Econophysics/Project/TwitterMonitor/Streamer_USA/UserIDs/user_ids.npy", np.array(list(self.user_ids))) # PIETRO 2
        if self.tweets_arrived%10:
            with open(self.save_path, "ab") as f:
                pickle.dump(self.tweets,f) # RICCARDO
                self.tweets  = []
                f.close()
        #np.save("C:\Users\Utente\Desktop\Progetti\Python\Econophysics\Project\TwitterMonitor\Streamer_USA\UserIDs\user_ids.npy", np.array(list(self.user_ids))) # DAVIDE
        print("Arrived",self.tweets_arrived, "tweets from", len(self.users_ids),"unique users", end = "\r")
    def keep_alive(self):
        #print("keep alive arrived")
        self.keep_alive_arrived += 1
        
    def on_error(self, status_code):
        print("Error", status_code)
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False
        return False

## Initialization

In [3]:
# Initialize user ids set 
##user_ids = set() # EMPTY
##user_ids = set(np.load("/Users/Pit/GitHub/Econophysics/Project/TwitterMonitor/Streamer_USA/UserIDs/user_ids.npy", allow_pickle=True)) # PIETRO 1 
##user_ids = set(np.load("/Users/pietromonticone/github/Econophysics/Project/TwitterMonitor/Streamer_USA/UserIDs/user_ids.npy", allow_pickle=True)) # PIETRO 2
##user_ids = set(np.load(r"C:\Users\Utente\Desktop\Progetti\Python\Econophysics\Project\TwitterMonitor\Streamer_USA\UserIDs\user_ids.npy", allow_pickle=True)) # DAVIDE 
#user_ids = set(np.load(r"C:\Users\39345\Documents\GitHub\Econophysics\Project\TwitterMonitor\Streamer_USA\UserIDs\ok_users.txt", allow_pickle=True)) # RICCARDO 
with open(r"F:\TwitterUsers\archiveTeam\processed_files\ok_users.txt", "rb") as f:
    ok_users = list(set(pickle.load(f)))
    f.close()
    
print(len(ok_users))

# Create a Stream 
myStreamListener = MyStreamListener(run_name = "users_id_track&follow",save_path =r"F:\TwitterUsers\archiveTeam\Streamer_archiveTeam" )

# Log an app 
app = Ec.logger(cr.apps[31], mode="user", wait_on_rate_limit=True)

# Create streamer 

def stream():
    i = 0
    while True:
        try: 
            myStream = tweepy.Stream(auth=app.auth, listener=myStreamListener)
            
            myStream.filter(
                follow = [str(ID) for ID in ok_users[:5000] ],
        
                languages=["en"], 
        
                is_async=False,
        
                stall_warnings=True)
            
            
        except Exception as e: 
            i += 1
            #print(i)
            #Relogging in 10 seconds 
            print("Relogging in", (2**i)*60 +1, " seconds!")
            print(e.__dict__)
            raise e
            time.sleep((2**i)*60 +1)
           
            continue
            #break

154824


## Streaming

In [4]:
#                 track=
#                 ["Vote2020", "vote2020", "election", "Election2020", "ELECTION2020","PresidentialElection2020", "election2020", "PresidentialElections", "2020Elections", "election2020", "2020Election",
#     "VoteTrumpOut2020", "VoteBidenOut2020", "TrumpSupporter", "BidenSupporter",
#     "voteblue2020", "VoteBlue", "VoteBlueToSaveAmerica", "votered2020", "VoteRed", "VoteRedToSaveAmerica",
#     "bidenharris",  "bidenharris2020", "trumppence", "trumppence2020", "BidenHarris2020", "bidenharris2020", "TrumpPence2020", "trumppence2020",
#     "KamalaHarris", "MikePence", "DonaldTrump", "JoeBiden", "trump2020", "biden2020", "joebiden", "donaldtrump", "GoTrump", "GoBiden",
#     "BidenForPresident2020", "TrumpForPresident2020",
#     "Democrats2020", "Republicans2020",
#     "GoTrump", "GoBiden", "BidenLies", "TrumpLies", 
#     "VoteJoeBiden", "voteJoeBiden", "VoteDonaldTrump", "voteDonaldTrump", 
#     "Biden2020", "JoeBiden2020", "Trump2020", "DonaldTrump2020", 
#     "PresidentBiden", "PresidentTrump",
#     "RepublicansForBiden", "RepublicansForTrump", "DemocratsForBiden", "DemocratsForTrump",
#     "NeverTrump", "NeverBiden",
#     "TrumpPence2020ToSaveAmerica", "BidenHarris2020ToSaveAmerica", "TRUMP2020ToSaveAmerica", "BIDEN2020ToSaveAmerica",
#     "Trump2020Victory", "Biden2020Victory", 
#     "BidenHarrisToSaveAmerica2020", "TrumpPenceToSaveAmerica2020",
#     "TeamJoeKamala2020", "TeamDonaldMike2020",
#     "VoteForJoeBiden", "VoteForDonaldTrump", "voteJoeBiden", "voteDonaldTrump",
#     "LibertariansForBiden", "LibertariansForTrump",
#     "TrumpIsOurPresident", "BidenIsOurPresident", "TrumpMustLose", "BidenMustLose",
#     "unfollowTrump", "UnfollowTrump", "unfollowBiden", "UnfollowBidem",
#     "VoteOutEveryDemocrat", "VoteOutEveryRepublican",
#     "voteforrepublican", "votefordemocrat", "VoteRedOnly", "VoteBlueOnly", "VoteRepublican", "VoteDemocrat",
#     "voteblue2020saveamerica", "votered2020saveamerica", "VoteTrump2020LetsSaveAmerica", "VoteTrump2020LetsSaveAmerica"
#                 ]

In [ ]:
stream()

# Check resource consumption
###app.rate_limit_status()["resources"]